In [22]:
pip uninstall kafka -y

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install kafka-python

Defaulting to user installation because normal site-packages is not writeable
  Using cached kafka_python-2.0.2-py2.py3-none-any.whl (246 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip uninstall kafka-python -y

Found existing installation: kafka-python 2.0.2
Uninstalling kafka-python-2.0.2:
  Successfully uninstalled kafka-python-2.0.2
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Conector
import os
## Atention to the version
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1 pyspark-shell'

In [2]:
# Importa o findspark e inicializa
import findspark
findspark.init()

# Import required modules
import pyspark
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, BooleanType
from pyspark.sql.functions import col, sum, from_json
import pyspark.sql.functions as F
from pyspark.sql.functions import *

In [3]:
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

24/05/24 17:42:17 WARN Utils: Your hostname, cj resolves to a loopback address: 127.0.1.1; using 192.168.15.34 instead (on interface enp2s0)
24/05/24 17:42:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/cj/.ivy2/cache
The jars for the packages stored in: /home/cj/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b601b2ea-e989-4bec-9e81-498f454a2f6c;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 511ms :: artifacts dl 15ms
	:: modules

In [4]:
# A especificação do esquema é necessária apenas para json e csv. Parquet, Delta e orc devem ficar bem sem ele
columns = StructType([
    StructField( "Firstname" ,StringType(), True ),\
    StructField( "Lastname" ,StringType(), True ),\
    StructField( "Jedi_Rank" , StringType(), True ),\
    StructField( "IsCouncilMember" , BooleanType(), True ),\
    StructField( "snapshot_date" , StringType(), True )
])
data = ([
    ("Obi", "Wan", "knight", True, "20224-05-20"),
    ("Anakin", "Skywlker", "Padawan", True, "20224-05-20"),
    ("Mace", "Windu", "Master", True, "20224-05-20")
])

In [5]:
df = spark.createDataFrame(data, columns)

In [6]:
df.show()
print(type(df)) 

+---------+--------+---------+---------------+-------------+
|Firstname|Lastname|Jedi_Rank|IsCouncilMember|snapshot_date|
+---------+--------+---------+---------------+-------------+
|      Obi|     Wan|   knight|           true|  20224-05-20|
|   Anakin|Skywlker|  Padawan|           true|  20224-05-20|
|     Mace|   Windu|   Master|           true|  20224-05-20|
+---------+--------+---------+---------------+-------------+

<class 'pyspark.sql.dataframe.DataFrame'>


In [7]:
tdf = df.withColumn("timestamp", current_timestamp())

In [26]:
kafka_stream = tdf.selectExpr("to_json(struct(*)) AS value") \
    .write \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("topic", "vandas-deshboard-gold") \
    .option("checkpointLocation", "../") \
    .save()